In [1]:
#Murat Tunç Advanced_Lane_Lines_Project_2.ipynb
#importing some useful packages
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import pickle
%matplotlib inline
##############################################################################################################

##############################################################################################################        
#Step-1:Camera Calibration,Find Object POints and Image Points
def Calibration(nx,ny):
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001) # termination criteria
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0) objp = np.zeros((6*7,3), np.float32)
    objp = np.zeros((ny*nx,3), np.float32)
    objp[:,:2] = np.mgrid[0:nx,0:ny].T.reshape(-1,2)
    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d point in real world space
    imgpoints = [] # 2d points in image plane.
    images = Chessboard_images = glob.glob('camera_cal\calibration*.jpg')# Read calibration images
    Distorted_Cheesboard_image = mpimg.imread('camera_cal\calibration1.jpg')
    img_size = (Distorted_Cheesboard_image.shape[1], Distorted_Cheesboard_image.shape[0])
    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        # Find the chess board corners
        ret, corners = cv2.findChessboardCorners(gray, (nx,ny),None)
        # If found, add object points, image points (after refining them)
        if ret == True:
            objpoints.append(objp)
            cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
            imgpoints.append(corners)
                
    # Do camera calibration
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size,None,None)
    return mtx,dist
##############################################################################################################

mtx,dist=Calibration(nx=9,ny=6)

# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # New line
        self.new_line = True
        # was the line detected in the last iteration?
        self.non_detect_tally = 0
        self.non_detect_tally_max = 3
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        # load some default values
        self.best_fit = []
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None     
        self.ploty = []
        
        #Source and Destion points
        self.src = np.float32(
                    [[590, 450],
                    [690,450],
                    [1060,650],
                    [300,650]])
        self.dst = np.float32(
                    [[250,0],
                    [896,0],
                    [896,720],
                    [250,720]])
        self.M    = cv2.getPerspectiveTransform(self.src,self.dst)
        self.Minv = cv2.getPerspectiveTransform(self.dst,self.src)
        self.ym_per_pix = 30/720.0 # meters per pixel in y dimension
        self.xm_per_pix = 3.7/700.0 # meters per pixel in x dimension

    #########################################################################################################

    def Binary_Conversion_Yellow_White(self,image):

        def bin_it(image, threshold):
            output_bin = np.zeros_like(image)
            output_bin[(image >= threshold[0]) & (image <= threshold[1])]=1
            return output_bin
        # convert image to hls colour space
        hls = cv2.cvtColor(image, cv2.COLOR_RGB2HLS).astype(np.float)
        # binary threshold values
        bin_thresh = [20, 255]
        # rgb thresholding for yellow
        lower = np.array([225,180,0],dtype = "uint8")
        upper = np.array([255, 255, 170],dtype = "uint8")
        mask = cv2.inRange(image, lower, upper)
        rgb_y = cv2.bitwise_and(image, image, mask = mask).astype(np.uint8)
        rgb_y = cv2.cvtColor(rgb_y, cv2.COLOR_RGB2GRAY)
        rgb_y = bin_it(rgb_y, bin_thresh)
        # rgb thresholding for white (best)
        lower = np.array([100,100,200],dtype = "uint8")
        upper = np.array([255, 255, 255],dtype = "uint8")
        mask = cv2.inRange(image, lower, upper)
        rgb_w = cv2.bitwise_and(image, image, mask = mask).astype(np.uint8)
        rgb_w = cv2.cvtColor(rgb_w, cv2.COLOR_RGB2GRAY)
        rgb_w = bin_it(rgb_w, bin_thresh) 
        # hls thresholding for yellow
        lower = np.array([20,120,80],dtype = "uint8")
        upper = np.array([45, 200, 255],dtype = "uint8")
        mask = cv2.inRange(hls, lower, upper)
        hls_y = cv2.bitwise_and(image, image, mask = mask).astype(np.uint8)
        hls_y = cv2.cvtColor(hls_y, cv2.COLOR_HLS2RGB)
        hls_y = cv2.cvtColor(hls_y, cv2.COLOR_RGB2GRAY)
        hls_y = bin_it(hls_y, bin_thresh)
        im_bin = np.zeros_like(hls_y)
        im_bin [(hls_y == 1)|(rgb_y==1)|(rgb_w==1)]= 1
        
        return im_bin
     ##############################################################################################################

    def display_on_frame(self,image, left_curverad, right_curverad, car_off):

        # create display texts on image
        font = cv2.FONT_HERSHEY_COMPLEX
        curve_disp_txt = 'Right_Curvature:' + str(np.round(right_curverad,2)) + '(m),Left_Curvature:' + str(np.round(left_curverad,2)) + '(m)'
        off_disp_txt = 'Vechile is  ' + str(np.round(car_off,2)) + 'm left of Center'
        cv2.putText(image, curve_disp_txt, (30, 60), font, 1, (0,0,0), 2)
        cv2.putText(image, off_disp_txt, (30, 110), font, 1, (0,0,0), 2)
        return image
    
    def find_lane_pixels(self, binary_warped):
   
        # Create an output image to draw on and  visualize the result
        out_img = np.dstack((binary_warped, binary_warped, binary_warped)).astype(np.int8)#*255
        window_img = np.zeros_like(out_img)
        # Take a histogram of the bottom half of the image
        histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
        # Find the peak of the left and right halves of the histogram
        # These will be the starting point for the left and right lines 
        midpoint = np.int(histogram.shape[0]//2)
        leftx_base = np.argmax(histogram[:midpoint])
        rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    
        # Create an output image to draw on and  visualize the result
        out_img = np.dstack((binary_warped, binary_warped, binary_warped)).astype(np.int8)#*255
        window_img = np.zeros_like(out_img)
    
        # Choose the number of sliding windows
        nwindows = 9
        # Set the width of the windows +/- margin
        margin = 100
        # Set minimum number of pixels found to recenter window
        minpix = 50
        # Set height of windows - based on nwindows above and image shape
        window_height = np.int(binary_warped.shape[0]/nwindows)
        # Identify the x and y positions of all nonzero pixels in the image
        nonzero = binary_warped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
    
        # Current positions to be updated later for each window in nwindows
        leftx_current = leftx_base
        rightx_current = rightx_base

        # Create empty lists to receive left and right lane pixel indices
        left_lane_inds = []
        right_lane_inds = []
        # Step through the windows one by one
        for window in range(nwindows):
            # Identify window boundaries in x and y (and right and left)
            win_y_low = binary_warped.shape[0] - (window+1)*window_height
            win_y_high = binary_warped.shape[0] - window*window_height
            win_xleft_low = leftx_current - margin
            win_xleft_high = leftx_current + margin
            win_xright_low = rightx_current - margin
            win_xright_high = rightx_current + margin
        
        
    
            # Draw the windows on the visualization image
            #cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
            #cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        
        
            # Identify the nonzero pixels in x and y within the window #
            good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high)  & (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
            good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        
            # Append these indices to the lists
            left_lane_inds.append(good_left_inds)
            right_lane_inds.append(good_right_inds)
        
        
            # If you found > minpix pixels, recenter next window on their mean position
            if len(good_left_inds) > minpix:
                leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
            if len(good_right_inds) > minpix:        
                rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

        # Concatenate the arrays of indices (previously was a list of lists of pixels)
        try:
            left_lane_inds = np.concatenate(left_lane_inds)
            right_lane_inds = np.concatenate(right_lane_inds)
        except ValueError:
            # Avoids an error if the above is not implemented fully
            pass
    
        #left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin))) 
        #right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))  

        # Extract left and right line pixel positions
        leftx = nonzerox[left_lane_inds]
        lefty = nonzeroy[left_lane_inds] 
        rightx = nonzerox[right_lane_inds]
        righty = nonzeroy[right_lane_inds]
    
        # Fit a second order polynomial to each using `np.polyfit`
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)
    
    
        # Generate x and y values for plotting
        ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0])
    
    
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    
        left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin))) 
        right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))  


        # change the colour of nonzero pixels
        out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [100, 0, 0]
        out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 100]
    
    
        # Generate a polygon to illustrate the search window area
        # And recast the x and y points into usable format for cv2.fillPoly()
        left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
        left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, ploty])))])
        left_line_pts = np.hstack((left_line_window1, left_line_window2))
        right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
        right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, ploty])))])
        right_line_pts = np.hstack((right_line_window1, right_line_window2))
    
        # Define y-value where we want radius of curvature
        # We choose the maximum y-value, corresponding to the bottom of the image
        y_eval = np.max(ploty)
        
        # Define conversions in x and y from pixels space to meters
        # Fit new polynomials to x,y in world space
        left_fit_cr = np.polyfit(ploty*self.ym_per_pix, left_fitx*self.xm_per_pix, 2)
        right_fit_cr = np.polyfit(ploty*self.ym_per_pix, right_fitx*self.xm_per_pix, 2)
        
        # Calculate the new radii of curvature
        left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*self.ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
        right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*self.ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    
        #fit_cr = np.polyfit(ploty * self.ym_per_pix, left_fitx * xm_per_pix, 2)
        #R_curvature = ((1 + (2*fit_cr[0]*y_eval*self.ym_per_pix + fit_cr[1])**2)**1.5) / np.absolute(2*fit_cr[0])
    
        # Draw the lane onto the warped blank image
        cv2.fillPoly(window_img, np.int_([left_line_pts]), (255,255, 0))
        cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    
        result = cv2.addWeighted(out_img, 1, window_img, 0.2, 0) 

        # %% Warp the detected lane boundaries back onto the original image
        # # Recast the x and y points into usable format for cv2.fillPoly()
        pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
        pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
        pts = np.hstack((pts_left, pts_right))
    
        # Draw the lane onto the warped blank image
        cv2.fillPoly(result, np.int_([pts]), (0,255, 0))
    
        return result, left_curverad, right_curverad,left_fit,right_fit  
    
    ##############################################################################################################
    #Step 7:The Result
    def pipeline(self, Distorted_test_image):
    
        img_size=(Distorted_test_image.shape[1], Distorted_test_image.shape[0])
        frame = cv2.undistort(Distorted_test_image, mtx, dist, None, mtx)
        original_frame = np.copy(frame)
    
        #Perspective Transform
        frame = cv2.warpPerspective(frame, self.M, img_size, flags=cv2.INTER_LINEAR)
        frame=self.Binary_Conversion_Yellow_White(frame)
        #Find Lane lines
        frame, left_curverad, right_curverad ,left_fit,right_fit = self.find_lane_pixels(frame)
    
        img_size = (frame.shape[:2][0], frame.shape[:2][1])
        # Warp the blank back to original image space using inverse perspective matrix (self.Minv)
        newwarp = cv2.warpPerspective(frame.astype(np.float32), self.Minv, img_size[::-1]) 
        newwarp = np.uint8(newwarp)
        # Combine the result with the original image
        frame = cv2.addWeighted(original_frame, 1, newwarp, 0.75, 0)
    
        left_intcpt = left_fit[0]*img_size[0]**2 + left_fit[1]*img_size[0] + left_fit[2]

        # right line intercept on x axis
        right_intcpt = right_fit[0]*img_size[0]**2 + right_fit[1]*img_size[0] + right_fit[2]

        lane_width = abs(right_intcpt-left_intcpt)
        lane_mid = (left_intcpt + right_intcpt)/2.0
        # car offset, im meters
        car_off = (lane_mid - img_size[1]/2.0)*self.xm_per_pix
        # display visuals on frame
        frame = self.display_on_frame(frame, left_curverad=left_curverad, right_curverad=right_curverad,
                        car_off=car_off)
        ret_frame = frame
        return ret_frame #.astype(np.uint8)


    ##############################################################################################################

            
Frame_lane_line = Line()

#Video Processing...
from moviepy.editor import VideoFileClip
from IPython.display import HTML
white_output='Project_2.mp4'
clip1 = VideoFileClip("project_video.mp4")
clip1.reader.close()
clip1.audio.reader.close_proc()
white_clip = clip1.fl_image(Frame_lane_line.pipeline)
%time white_clip.write_videofile(white_output,audio=False)





[MoviePy] >>>> Building video Project_2.mp4
[MoviePy] Writing video Project_2.mp4


100%|█████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [05:56<00:00,  3.06it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: Project_2.mp4 

Wall time: 5min 59s
